In [1]:
import pandas as pd  
import os

In [4]:
res = pd.read_csv("experiments.csv")
res.tail(18)

,name,accuracy,f1,confusion_matrix,training_time,epochs,min_loss,datetime,MAF method,dataset,breaks,mult_rules,all_losses,clustering_alg,label_for_dist
105,"dataset=Brain Tumor, label_for_dist=labels, cl...",0.966667,0.956522,[[18 1]\n [ 0 11]],1.921135,77,0.000329,27-04-24 23:57,clustering,Brain Tumor,3.0,False,"[0.18845222890377045, 0.17826935648918152, 0.1...",kmeans,labels
106,"dataset=Brain Tumor, label_for_dist=labels, cl...",0.933333,0.916667,[[17 2]\n [ 0 11]],3.706414,143,0.009628,27-04-24 23:57,random,Brain Tumor,3.0,False,"[0.2513660192489624, 0.23288555443286896, 0.21...",kmeans,labels
107,"dataset=breast-cancer-wisconsin, label_for_dis...",1.000000,1.000000,[[17 0]\n [ 0 13]],1.915689,54,0.015355,27-04-24 23:57,clustering,breast-cancer-wisconsin,3.0,False,"[0.11070981621742249, 0.102330282330513, 0.094...",kmeans,labels
108,"dataset=breast-cancer-wisconsin, label_for_dis...",0.966667,0.960000,[[17 0]\n [ 1 12]],8.365853,203,0.013834,27-04-24 23:57,random,breast-cancer-wisconsin,3.0,False,"[0.2783694565296173, 0.2583649456501007, 0.239...",kmeans,labels
109,"dataset=uniform_df, label_for_dist=labels, clu...",0.966667,0.968750,[[27 2]\n [ 0 31]],6.386958,159,0.012744,28-04-24 0:02,clustering,uniform_df,3.0,False,"[0.2357240617275238, 0.22866366803646088, 0.22...",kmeans,labels
110,"dataset=uniform_df, label_for_dist=labels, clu...",0.966667,0.968750,[[27 2]\n [ 0 31]],11.786183,287,0.013378,28-04-24 0:03,random,uniform_df,3.0,False,"[0.2552374303340912, 0.25232088565826416, 0.24...",kmeans,labels
111,"dataset=uniform_df, label_for_dist=labels, clu...",0.966667,0.972973,[[11 0]\n [ 1 18]],3.039772,143,0.019417,28-04-24 0:03,clustering,uniform_df,3.0,False,"[0.2540368437767029, 0.24707405269145966, 0.24...",kmeans,labels
112,"dataset=uniform_df, label_for_dist=labels, clu...",0.966667,0.972973,[[11 0]\n [ 1 18]],7.475388,285,0.021116,28-04-24 0:03,random,uniform_df,3.0,False,"[0.2564598619937897, 0.2536381483078003, 0.250...",kmeans,labels
113,"dataset=Brain Tumor, label_for_dist=labels, cl...",0.981399,0.979512,[[606 5]\n [ 16 502]],226.094487,117,0.017742,28-04-24 0:08,clustering,Brain Tumor,3.0,False,"[0.18103890120983124, 0.1676415055990219, 0.15...",kmeans,labels
114,"dataset=Brain Tumor, label_for_dist=labels, cl...",0.984942,0.983415,[[608 3]\n [ 14 504]],345.846017,141,0.026200,28-04-24 0:14,random,Brain Tumor,3.0,False,"[0.2958773076534271, 0.2769116163253784, 0.258...",kmeans,labels
